In [1]:
from easydict import EasyDict as edict

args = edict()

args.tmp_dir = "Cell_Phones_and_Accessories"
args.dataset = "Cell_Phones_and_Accessories"

# KG related files
args.kg_users_file = '{}/kg_users_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_items_file = '{}/kg_items_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_entities_file = '{}/kg_entities_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_relations_file = '{}/kg_relations_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_train_triples_file = '{}/kg_train_triples_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_train_gt_file = '{}/kg_train_gt_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_other_triples_file = '{}/kg_other_triples_{}.txt'.format(args.tmp_dir, args.dataset)

args.transE_embedding_file = '{}/openke/transe_epoch_100_lr_0.002.ckpt'.format(args.tmp_dir, args.dataset)

In [2]:
from os import defpath
#from utils import args
import numpy as np
from collections import defaultdict
#import torch
from tqdm import tqdm

#%%
class KnowledgeGraph2():
    def __init__(self, kg_file, purchase_file):
        self.kg_file = kg_file
        self.purchase_file = purchase_file
        self.G = defaultdict(list)
        self.uip = defaultdict(list) # user purchased item
        self.ui = defaultdict(list)
        self._load_kg()


    def _load_kg(self):
        with open(self.kg_file, 'r') as f:
            for line in f:
                k, v, r = map(int, line.rstrip().split('\t'))
                self.G[k].append((v,r))
                self.G[v].append((k,r)) # reverse edge
        with open(self.purchase_file, 'r') as f:
            for line in f:
                k, v, r = map(int, line.rstrip().split('\t'))
                self.uip[k].append((v,r))
                self.ui[k].append(v)
                self.G[v].append((k,r)) # reverse edge

        # remove repeated edges
        for k in self.G:
            self.G[k] = list(set(self.G[k]))
        for k in self.uip:
            self.uip[k] = list(set(self.uip[k]))
        for k in self.ui:
            self.ui[k] = list(set(self.ui[k]))

    def get_k_hop_path(self, u, k):
        path = []
        items = set(self.ui[u])
        to_visit = [self.G[u].copy()]
        current_path = [(u, -1)]
        d = 1
        while True:
            if d == k:
                for u,i in to_visit[-1]:
                    if u in items:
                        path.append(current_path+[(u,i)])
                current_path.pop()
                to_visit.pop()
                d -= 1
            try:
                u, p = to_visit[-1].pop()
                # if u, p == 149744, 32:
                #     print(current_path)
            except IndexError:
                d-=1
                try:
                    to_visit.pop()
                except IndexError:
                    break
                current_path.pop()
                continue
            current_path.append((u, p))
            # to_visit.append(self.G[u].copy()+self.uip[u].copy())
            # to_visit.append(set(list(zip(*(self.G[u].copy()+self.uip[u].copy())))[0]) - set(list(zip(*current_path)))[0])
            to_visit.append([])
            for uu, pp in set(self.G[u].copy()+self.uip[u].copy()):
                if uu not in set(list(zip(*current_path))[0]):
                    to_visit[-1].append((uu, pp))

            d += 1
        return path

In [3]:
KG = KnowledgeGraph2(args.kg_other_triples_file, args.kg_train_triples_file)

In [110]:
KG.ui[0]

NameError: name 'KG' is not defined

In [6]:
from threading import Thread
import pickle

In [33]:
def f(i):
    b = []
    u = list(KG.ui.keys())[i::k]
    for uu in u:
        a = KG.get_k_hop_path(uu,3)
        if a:
            b.append(a)
    return b

In [34]:
from time import time
from  multiprocessing import Process,Pool
k=8
list(KG.ui.keys())[::k]
t = []
result = []
start = time()
pool = Pool(k)
for i in range(k):
    #t.append(Thread(target=f, args=(list(KG.ui.keys())[i::k])))
    t.append(pool.apply_async(func=f, args=(i,)))
    #t[-1].setDaemon(True)

pool.close()
pool.join()
for tt in t:
    result += tt.get()
print(time()-start)

15543.388068199158


In [38]:
15543/60/60

4.3175

In [7]:
from time import time
k=6
list(KG.ui.keys())[::k]
t = []
result = []
start = time()
for i in range(k):
    #t.append(Thread(target=f, args=(list(KG.ui.keys())[i::k])))
    t.append(Thread(target=f, args=(i,)))
    #t[-1].setDaemon(True)
    t[-1].start()

# for tt in t:
#     tt.join()
print(time()-start)
# for tt in t:
#     result = result + tt.getResult()

0.85756516456604


In [109]:
len(result)

60292

In [88]:
len(result[135])

67

In [37]:
with open('all_path.txt','wb') as fp:
    pickle.dump(result, fp)

In [7]:
with open('all_path.txt','rb') as fp:
    result = pickle.load(fp)

In [14]:
import torch
import itertools

In [15]:
ckpt = torch.load(args.transE_embedding_file, map_location=torch.device('cpu'))
embeds = ckpt['ent_embeddings.weight'].cpu().numpy()
rels = ckpt['rel_embeddings.weight'].cpu().numpy()
print(type(embeds), embeds.shape)
print(type(rels), rels.shape)

<class 'numpy.ndarray'> (169331, 100)
<class 'numpy.ndarray'> (45, 100)


In [10]:
chain = itertools.chain(*result)
re = list(chain)

In [13]:
re = np.array(re, dtype=int)
re.shape

(18848774, 4, 2)

In [91]:
re[0]

array([[     0,     -1],
       [160426,     10],
       [ 10157,     10],
       [ 90652,      0]])

In [22]:
nodes = re[:,:,0]
paths = re[:,:,1]

In [92]:
dist = embeds[nodes[:,1:]] - embeds[nodes[:,:-1]] - rels[paths[:,1:]] # a->b  a+r=b
dist = np.sum(np.linalg.norm(dist,ord=1, axis=-1), axis=-1) ###L1

In [93]:
dist.shape

(18848774,)

In [94]:
dist

array([47.421547, 46.093384, 46.798183, ..., 49.559258, 55.071545,
       47.326683], dtype=float32)

In [95]:
import numpy_indexed as npi
ui, idx = npi.group_by(re[:,[0,3],0]).argmin(dist)

In [96]:
opti_paths = re[idx]

In [97]:
def represent_avg(v):
    return np.average(v, axis=-2)

In [98]:
opti_paths.shape

(321053, 4, 2)

In [99]:
opti_paths[0]

array([[     0,     -1],
       [115820,      1],
       [ 10164,      1],
       [ 68872,      0]])

In [100]:
np.save('inter_data/opti_paths.npy',opti_paths)

In [101]:
nodes_emb = embeds[opti_paths[:,:,0]]
edges_emb = rels[opti_paths[:,1:,1]]

In [102]:
opti_paths_emb = np.concatenate([nodes_emb, edges_emb],axis=1)

In [103]:
rep = represent_avg(opti_paths_emb)

In [104]:
rep.shape

(321053, 100)

In [105]:
np.save('inter_data/opti_path_rep.npy',rep)

# model.py

In [83]:
opti_paths[:,[0,-1],0]

array([[    0, 68872],
       [    0, 90652],
       [    0, 99801],
       ...,
       [60927, 95618],
       [60927, 92038],
       [61183, 87825]])